In [1]:
!pip install -U bitsandbytes>=0.41.0



In [2]:
!pip install -U transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.5.2
    Uninstalling accelerate-1.5.2:
      Successfully uninstalled accelerate-1.5.2


In [3]:
!pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 123.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 955.6/

In [4]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 132.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.0 MB/s eta 0:00:00


In [6]:

import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import gradio as gr

# Initialize the language model
model_name = "NousResearch/Hermes-2-Pro-Mistral-7B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True
)

chatbot_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

# Patient data functions
def get_patient_info(patient_id):
    return {
        "name": "Massoud DIALLO",
        "exercises_progress": "50% d'exercices complétés",
        "mental_state": "Stress élevé",
        "mental_state_score": 75,
    }

def init_patient_info(patient_id):
    patient_info = get_patient_info(patient_id)
    return (
        patient_info["name"],
        patient_info["exercises_progress"],
        patient_info["mental_state"],
        patient_info["mental_state_score"]
    )

# Chat function
def respond(message, history):
    prompt = f"""<|im_start|>system
    You are a French mental health assistant. Provide concise, supportive responses (2-3 sentences max).
    Focus on active listening and practical suggestions. Respond in French.<|im_end|>
    <|im_start|>user
    {message}<|im_end|>
    <|im_start|>assistant
    """

    output = chatbot_pipeline(
        prompt,
        max_new_tokens=150,
        temperature=0.7,
        do_sample=True,
        top_k=50,
        top_p=0.95
    )

    response = output[0]['generated_text'].split("<|im_start|>assistant")[-1]
    response = response.split("<|im_end|>")[0].strip()
    return response

# Create the interface
with gr.Blocks(theme=gr.themes.Soft()) as patient_chat_page:
    with gr.Row():
        gr.Markdown("<h1 style='text-align: center;'>🧠 MindScape UC AI- Espace Patient</h1>")

    with gr.Row():
        # Left column (chat)
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(
                label="Conversation avec MindScape",
                bubble_full_width=False,
                avatar_images=(
                    "https://img.freepik.com/vecteurs-libre/conception-modele-logo-communication_23-2149881439.jpg",
                    "https://t3.ftcdn.net/jpg/03/35/16/66/360_F_335166628_b2M3WgWbbZqxNHsRt6ZxHzk1dtCrWhVx.jpg"
                )
            )

            with gr.Row():
                user_input = gr.Textbox(
                    placeholder="Écrivez votre message...",
                    show_label=False,
                    container=False
                )
                send_btn = gr.Button("Envoyer", variant="primary")

        # Right column (patient info)
        with gr.Column(scale=1, min_width=250):
            name, progress, mental_state, mental_state_score = init_patient_info(1)

            with gr.Accordion("📋 Profil Patient", open=True):
                gr.Markdown(f"Nom : {name}  \n*Progrès :* {progress}")

                with gr.Group():
                    gr.Markdown("### État Mental")
                    gr.Markdown(f"Niveau actuel : {mental_state}")
                    gr.Markdown(f"Score de stress : {mental_state_score}/100")

                    if mental_state_score > 80:
                        status = "🔴 Stress élevé - Consultez un spécialiste"
                    elif mental_state_score > 50:
                        status = "🟠 Stress modéré - Exercices recommandés"
                    else:
                        status = "🟢 Stress faible - Bon équilibre"
                    gr.Markdown(f"Recommandation : {status}")

    # Event handlers
    def chat_fn(message, chat_history):
        if not message:
            return "", chat_history
        bot_response = respond(message, chat_history)
        chat_history.append((message, bot_response))
        return "", chat_history

    send_btn.click(
        chat_fn,
        [user_input, chatbot],
        [user_input, chatbot]
    )

    user_input.submit(
        chat_fn,
        [user_input, chatbot],
        [user_input, chatbot]
    )

patient_chat_page.launch()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/643 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/2.68G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-6-3c9aede3b774>:71: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
<ipython-input-6-3c9aede3b774>:71: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://94b19f2cbd2e343ee5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import torch
import gc
from transformers import pipeline

# Clear the chatbot instance if it exists
if 'chatbot' in globals():
    del chatbot

# Clear PyTorch cache
torch.cuda.empty_cache()

# Run garbage collection
gc.collect()

print("Chatbot instance deleted and cache cleared successfully.")

Chatbot instance deleted and cache cleared successfully.
